In [1]:
import matplotlib.pyplot as plt
import json
import os
import sys
 
# Path for spark source folder
os.environ['SPARK_HOME'] = "/Users/jonneff/spark-1.4.0-bin-hadoop2.6"
 
# Append pyspark to Python Path
sys.path.append("/Users/jonneff/spark-1.4.0-bin-hadoop2.6/python")
 
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
# Load in the testing code and check to see if your answer is correct
# If incorrect it will report back '1 test failed' for each failed test
# Make sure to rerun any cell you change before trying the test again
from test_helper import Test
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [2]:
from tdigest import TDigest
from numpy.random import random
from operator import add

# 0.0 Read data

Read in data.  For now, on local filesystem.  LATER, ADD CODE TO READ FROM S3.  

In [3]:
df = sqlContext.read.json("/Users/jonneff/Desktop/DE/RC_2007-10")

In [4]:
df.count()

150429

In [5]:
df.dtypes

[('archived', 'boolean'),
 ('author', 'string'),
 ('author_flair_css_class', 'string'),
 ('author_flair_text', 'string'),
 ('body', 'string'),
 ('controversiality', 'bigint'),
 ('created_utc', 'string'),
 ('distinguished', 'string'),
 ('downs', 'bigint'),
 ('edited', 'string'),
 ('gilded', 'bigint'),
 ('id', 'string'),
 ('link_id', 'string'),
 ('name', 'string'),
 ('parent_id', 'string'),
 ('retrieved_on', 'bigint'),
 ('score', 'bigint'),
 ('score_hidden', 'boolean'),
 ('subreddit', 'string'),
 ('subreddit_id', 'string'),
 ('ups', 'bigint')]

In [6]:
df.printSchema()

root
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: long (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- retrieved_on: long (nullable = true)
 |-- score: long (nullable = true)
 |-- score_hidden: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- ups: long (nullable = true)



In [7]:
df.head(1)

[Row(archived=True, author=u'bostich', author_flair_css_class=None, author_flair_text=None, body=u'test', controversiality=0, created_utc=u'1192450635', distinguished=None, downs=0, edited=u'false', gilded=0, id=u'c0299an', link_id=u't3_5yba3', name=u't1_c0299an', parent_id=u't3_5yba3', retrieved_on=1427426409, score=1, score_hidden=False, subreddit=u'reddit.com', subreddit_id=u't5_6', ups=1)]

In [8]:
df.take(1)

[Row(archived=True, author=u'bostich', author_flair_css_class=None, author_flair_text=None, body=u'test', controversiality=0, created_utc=u'1192450635', distinguished=None, downs=0, edited=u'false', gilded=0, id=u'c0299an', link_id=u't3_5yba3', name=u't1_c0299an', parent_id=u't3_5yba3', retrieved_on=1427426409, score=1, score_hidden=False, subreddit=u'reddit.com', subreddit_id=u't5_6', ups=1)]

# 1.0 Find minimum comment timestamp for each post

For each post link_id, find minimum created_utc timestamp [can't trust data is ordered by time] and store in key-value pair (pair RDD) {link_id: min_created_utc},  (Plan B:  set up API to get timestamp for all posts in Reddit)

In [9]:
redditRDD = df.rdd

In [10]:
redditRDD.take(1)

[Row(archived=True, author=u'bostich', author_flair_css_class=None, author_flair_text=None, body=u'test', controversiality=0, created_utc=u'1192450635', distinguished=None, downs=0, edited=u'false', gilded=0, id=u'c0299an', link_id=u't3_5yba3', name=u't1_c0299an', parent_id=u't3_5yba3', retrieved_on=1427426409, score=1, score_hidden=False, subreddit=u'reddit.com', subreddit_id=u't5_6', ups=1)]

In [11]:
rRDD = df.map(lambda r: (r.id, (r.body, int(r.created_utc), r.link_id, r.parent_id, int(r.score), r.subreddit, r.subreddit_id)))

In [12]:
rRDD.take(1)

[(u'c0299an',
  (u'test', 1192450635, u't3_5yba3', u't3_5yba3', 1, u'reddit.com', u't5_6'))]

Sort to find minimum timestamp created_utc for each post link_id

In [13]:
rRDD.count()

150429

In [14]:
minTimeRDD = (rRDD.map(lambda (k, v): (v[2],v[1])) # maps to (link_id, created_utc)
                  .reduceByKey(lambda a, b:  a if a < b else b)
              )

In [15]:
minTimeRDD.take(1)

[(u't3_5yxsg', 1193154263)]

# 2.0 Filter to include only extreme up and down votes (top 3% of subreddit)

Filter to retain only records that are top or bottom 3% in comment score (upvotes-downvotes) of their subreddit.  Reduces dataset for all subsequent processing.

Find 3% and 97% percentiles for each subreddit.  Use T-digest data structure for highly accurate approximate percentiles: 
http://dataorigami.net/blogs/napkin-folding/19055451-percentile-and-quantile-estimation-of-big-data-the-t-digest

In [16]:
def digest(values):
    digest = TDigest()
    digest.batch_update(values)
    return digest

In [27]:
def digest1(value):
    digest = TDigest()
    digest.update(value)
    return digest

Need to get at scores for each subreddit, THEN map using digest.

tdigest with mapreduce doesn't work; i get something that is not a tdigest object.  Ronak suggests a solution, see project log.

In [34]:
subredditDigestRDD = (rRDD.map(lambda (k, v):  (v[5], v[4]))
                        .map(lambda (k, v): (k, digest1(v)))  
                        .reduceByKey(lambda a, b:  a + b)
                   )

In [35]:
type(subredditDigestRDD)

pyspark.rdd.PipelinedRDD

In [36]:
subredditDigest = subredditDigestRDD.collectAsMap()

In [37]:
type(subredditDigest['politics'])

tdigest.tdigest.TDigest

In [38]:
subredditDigest['politics'].percentile(3)

-5.3696052631578945

In [39]:
subredditDigest['politics'].percentile(97)

16.797647058823536

Alyssa's values for politics were (-6,

Map reddit RDD to key,value pair (subreddit, [score])
Reduce by adding list b to the end of list a.  This creates a list of scores by subreddit.
Create a t-digest for each subreddit representing distribution of scores.

FOLLOWING CODE DOESN'T WORK BECAUSE SPARK WON'T LET ME CAST V[4] AS A LIST AND WON'T RECOGNIZE [v[4]] as a list either.

subredditDigestRDD = (rRDD.map(lambda (k, v): (v[5], list(v[4])))
                         .reduceByKey(lambda a, b:  a.extend(b))
                      )
                       #   .map(lambda (k, v):  (k,digest(v)))
                      

In [26]:
print subredditDigestRDD.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 17.0 failed 1 times, most recent failure: Lost task 0.0 in stage 17.0 (TID 31, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/jonneff/spark-1.4.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/Users/jonneff/spark-1.4.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/jonneff/spark-1.4.0-bin-hadoop2.6/python/pyspark/rdd.py", line 2318, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/Users/jonneff/spark-1.4.0-bin-hadoop2.6/python/pyspark/rdd.py", line 2318, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/Users/jonneff/spark-1.4.0-bin-hado

In [19]:
subredditDigest = subredditDigestRDD.collectAsMap() # Turn RDD into dictionary

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 12.0 failed 1 times, most recent failure: Lost task 0.0 in stage 12.0 (TID 24, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/jonneff/spark-1.4.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/Users/jonneff/spark-1.4.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/jonneff/spark-1.4.0-bin-hadoop2.6/python/pyspark/rdd.py", line 2318, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/Users/jonneff/spark-1.4.0-bin-hadoop2.6/python/pyspark/rdd.py", line 2318, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/Users/jonneff/spark-1.4.0

In [57]:
polPerDigest = subredditDigest[u'politics']

In [59]:
type(polPerDigest)

builtin_function_or_method

Filter records to retain only top and bottom 3% of comment scores.

# 3.0 Filter out exclusions

Filter out exclusions.  Further reduces dataset.

# 4.0  Calculate timeSince

Calculate timeSince based on created_utc and min_created_utc from pair RDD.

# 5.0  Calculate commentLength

Clean comment body and calculate commentLength.

# 6.0 Run sentiment analysis and calculate posNegDiff

# 7.0 Set up OHE for subreddit

# 8.0 (Deal with sparse matrices from OHE if necessary; skip for now)

# 9.0 Run logistic regression

In [2]:
DATABASE = 'reddit'

In [3]:
engine = create_engine("mysql+pymysql://root@localhost/" + str(DATABASE))
con = engine.connect()
dataid = 1022

In [4]:
redditDB = con.execute("SELECT * FROM comments6")

In [5]:
reddit = pd.DataFrame(redditDB.fetchall())
reddit.columns = redditDB.keys()

In [6]:
reddit.head()

,commentAuthor,commentCreated,commentID,commentLink,commentScore,postID,subreddit,postAuthor,postBody,postCreated,...,NegRaw,vNegRaw,vPos,Pos,Neg,vNeg,posNegRatio,posNegDiff,commentBody2,commentLengthSW
0,Ajunadeeps,1432770000,crmzgnx,http://www.reddit.com/r/leagueoflegends/commen...,586,37ihzo,leagueoflegends,DBlackjack21,http://gfycat.com/HopefulGleefulHoiho,1432770000,...,0,0,None,None,None,None,0,0.600000,Best backdoor since gay porn.,5
1,DBlackjack21,1432780000,crn46mg,http://www.reddit.com/r/leagueoflegends/commen...,441,37ihzo,leagueoflegends,DBlackjack21,http://gfycat.com/HopefulGleefulHoiho,1432770000,...,0,0,None,None,None,None,0,0.200000,[Objection!](http://gfycat.com/OptimisticSkinn...,5
2,ThePowerOfAura,1432780000,crn58hk,http://www.reddit.com/r/leagueoflegends/commen...,325,37ihzo,leagueoflegends,DBlackjack21,http://gfycat.com/HopefulGleefulHoiho,1432770000,...,0,0,None,None,None,None,0,0.166667,"Dude and I thought Sion was easy to play, clea...",18
3,SkyhuntL,1432790000,crn9iqq,http://www.reddit.com/r/leagueoflegends/commen...,125,37ihzo,leagueoflegends,DBlackjack21,http://gfycat.com/HopefulGleefulHoiho,1432770000,...,0,0,None,None,None,None,NaN,0.000000,"Or a driving license, that is.",7
4,offlightsedge,1432790000,crnc5oz,http://www.reddit.com/r/leagueoflegends/commen...,33,37ihzo,leagueoflegends,DBlackjack21,http://gfycat.com/HopefulGleefulHoiho,1432770000,...,0,0,None,None,None,None,0,0.333333,Preparing for a wide turn like that looks like...,12


In [7]:
reddit['timeSince'].describe()

count     253519.000000
mean       32795.809746
std        57652.954846
min            0.000000
25%         9216.500000
50%        24702.000000
75%        43076.000000
max      3279309.000000
Name: timeSince, dtype: float64

In [8]:
reddit['commentScore'].describe()

count    287829.000000
mean          6.800795
std          34.962045
min        -416.000000
25%           0.000000
50%           1.000000
75%           4.000000
max        2143.000000
Name: commentScore, dtype: float64

In [9]:
redditOrig = reddit

In [10]:
redditA = reddit[['commentID', 'subreddit', 'commentScore', 'timeSince', 'commentLength', 'posNegDiff']]

In [11]:
def perc(subreddit):
    return (round(np.percentile(reddit[reddit['subreddit'] == subreddit]['commentScore'], 3, axis = 0)), round(np.percentile(reddit[reddit['subreddit'] == subreddit]['commentScore'], 97, axis = 0)))

In [12]:
LoLper = perc('leagueoflegends')
GGper = perc('GirlGamers')
picsper = perc('pics')
polper = perc('politics')

In [13]:
print(LoLper)
print(GGper)
print(picsper)
print(polper)

(-7.0, 66.0)
(0.0, 26.0)
(-5.0, 60.0)
(-6.0, 24.0)


In [14]:
def score(subreddit, per):
    data = redditA[redditA['subreddit'] == subreddit]
    data['scoreC'] = np.nan
    data['scoreC'][data['commentScore'] <= per[0]] = 0
    data['scoreC'][data['commentScore'] >= per[1]] = 1
    return data

In [15]:
LoL = score('leagueoflegends', LoLper)
GG = score('GirlGamers', GGper)
pic = score('pics', picsper)
pol = score('politics', polper)


/anaconda/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:3368: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data

In [16]:
print LoL['scoreC'].describe()
print GG['scoreC'].describe()
print pic['scoreC'].describe()
print pol['scoreC'].describe()

count    6256.000000
mean        0.468191
std         0.499027
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: scoreC, dtype: float64
count    499.000000
mean       0.386774
std        0.487500
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: scoreC, dtype: float64
count    5535.000000
mean        0.487986
std         0.499901
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: scoreC, dtype: float64
count    3763.000000
mean        0.484985
std         0.499841
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: scoreC, dtype: float64


In [17]:
redditB = pd.concat([LoL, GG, pic, pol])

In [18]:
redditB.groupby('scoreC').mean()

,commentScore,timeSince,commentLength,posNegDiff
scoreC,,,,
0,-16.081737,20443.331973,30.225986,-0.016371
1,142.337997,10461.994676,27.414966,0.005304


In [19]:
y, X = dmatrices('scoreC ~ C(subreddit) + timeSince + commentLength + posNegDiff', redditB, return_type = "dataframe")
print X.columns

Index([u'Intercept', u'C(subreddit)[T.leagueoflegends]', u'C(subreddit)[T.pics]', u'C(subreddit)[T.politics]', u'timeSince', u'commentLength', u'posNegDiff'], dtype='object')


In [20]:
X = X.rename(columns = {'C(subreddit)[T.leagueoflegends]': 'subLoL', 
                        'C(subreddit)[T.pics]': 'subPics', 
                        'C(subreddit)[T.politics]': 'subPol'})

In [21]:
y = np.ravel(y)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [23]:
model = LogisticRegression()
model = model.fit(X_train, y_train)

In [24]:
f = open('model', 'w')
cPickle.dump(model, f)
f.close()

In [25]:
pd.DataFrame(zip(X.columns, np.transpose(model.coef_)))

,0,1
0,Intercept,[0.410245670233]
1,subLoL,[0.0583930466606]
2,subPics,[0.244273666339]
3,subPol,[0.100347697994]
4,timeSince,[-7.30151157008e-05]
5,commentLength,[-0.000133795596042]
6,posNegDiff,[0.0322472638634]


In [26]:
predicted = model.predict(X_test)
print predicted

[ 1.  1.  1. ...,  0.  0.  1.]


In [27]:
probs = model.predict_proba(X_test)
print probs

[[ 0.26103584  0.73896416]
 [ 0.45698479  0.54301521]
 [ 0.49617659  0.50382341]
 ..., 
 [ 0.62978508  0.37021492]
 [ 0.79438198  0.20561802]
 [ 0.27561929  0.72438071]]


In [28]:
print metrics.accuracy_score(y_test, predicted)
print metrics.roc_auc_score(y_test, probs[:, 1])

0.646433632499
0.709111871693


In [29]:
print metrics.confusion_matrix(y_test, predicted)
print metrics.classification_report(y_test, predicted)

[[1400  806]
 [ 691 1337]]
             precision    recall  f1-score   support

        0.0       0.67      0.63      0.65      2206
        1.0       0.62      0.66      0.64      2028

avg / total       0.65      0.65      0.65      4234



In [30]:
model.predict_proba(np.array([1, 0, 0, 0, 2*60*60, 30, .001]))

array([[ 0.42780781,  0.57219219]])

In [31]:
model.predict_proba(np.array([1, 0, 0, 0, 10*60*60, 10, -.1]))

array([[ 0.85967768,  0.14032232]])

In [32]:
model.predict_proba(np.array([0, 0, 1, 0, 1*60*60, 10, 2]))

array([[ 0.38821503,  0.61178497]])